# Инфраструктура для моделей машинного обучения. Практическая работа

# Цель практической работы

Потренироваться в использовании библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в практическую работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.
10. Обратная связь.


# Что оценивается

- Пройдены все этапы работы.
- Спарк-сессия успешно запущена.
- Данные прочитаны.
- Все колонки с числовыми значениями преобразованы в числовые типы данных (Int или Double).
- Отфильтрованы все строки с Null-значениями.
- Созданы новые фичи.
- Все категориальные колонки преобразованы в числовой вид, выполнены все этапы векторизации признаков.
- Выборка разделена на обучающую и тестовую.
- Создано три объекта: модель, сетка гиперпараметров и evaluator.
- Создан объект класса CrossValidator и обучен на обучающей выборке.
- Выбрана лучшая модель, посчитана метрика качества лучшей модели.


# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

Инициализируйте спарк-сессию.

Эта ячейка нужна для того, чтобы заргузить необходимые библиотеки и настроить окружение Google Colab для работы со Spark.

Просто запустите её перед выполением задания :)

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySparkTutorial") \
    .getOrCreate()


# 2. Загрузка данных
Загрузите данные, сохраните их в переменную типа sparkDataframe, используя метод read.csv (не забывайте про header и delimiter).

In [26]:
from google.colab import files
uploaded = files.upload()


Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn.csv


In [112]:
# from google.colab import drive
# drive.mount("content/drive")

df = spark.read.option("header", True).option("delimiter", ",").csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# 3. Ознакомление с данными
1. Выведите на экран первые несколько строк датафрейма.


In [113]:
df.show(5,8)

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|  7590-...|Female|            0|    Yes|        No|     1|          No|     No ph...|            DSL|            No|         Yes|              No|         No|         No|             No|Month...|             Yes| 


2. Выведите общее количество строк датафрейма.



In [114]:
df.count()

7043

3. Выведите структуру (схему) датафрейма.

In [115]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразуйте тип столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохраните преобразованный датафрейм в новую переменную.

## Совет

Если вам сложно выполнить это задание, изучите дополнительные материалы: [об операторе Cast](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/), [об операторе Select](https://sparkbyexamples.com/pyspark/select-columns-from-pyspark-dataframe/).



In [116]:
from pyspark.sql.functions import expr, col

df_num = df.select(
    [col("tenure").cast("Int").alias("tenure"),
    col("MonthlyCharges").cast("Double").alias("MonthlyCharges"),
    col("SeniorCitizen").cast("Int").alias("SeniorCitizen"),
    col("TotalCharges").cast("Double").alias("TotalCharges")] +
    [col(c) for c in df.columns if c not in ["tenure", "MonthlyCharges", "TotalCharges", "SeniorCitizen"]]
)

df_num.printSchema()

root
 |-- tenure: integer (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- Churn: string (nullable = true)



# 5. Очистка данных
Проверьте, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Выведите на экран несколько строк с Null-значениями в одной из колонок.

Сохраните очищенный от строк с Null-значениями датафрейм в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Подумайте, как упроситить эту работу, если использовать, например, перебор с циклом for.

[Примеры использования операторов isNull() и isNotNull()](https://sparkbyexamples.com/pyspark/pyspark-isnull/).


In [117]:
for column in df_num.columns:
    null_rows = df_num.filter(col(column).isNull())
    if null_rows.count() > 0:
        print(f"Null-значения в колонке: {column}")
        null_rows.show(5)  # Показать первые 5 строк с Null-значениями

Null-значения в колонке: TotalCharges
+------+--------------+-------------+------------+----------+------+-------+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+-----+
|tenure|MonthlyCharges|SeniorCitizen|TotalCharges|customerID|gender|Partner|Dependents|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|Churn|
+------+--------------+-------------+------------+----------+------+-------+----------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+-----+
|     0|         52.55|            0|    

In [118]:
df_cleaned = df_num.na.drop()
df_cleaned.show(5)


+------+--------------+-------------+------------+----------+------+-------+----------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+-----+
|tenure|MonthlyCharges|SeniorCitizen|TotalCharges|customerID|gender|Partner|Dependents|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|Churn|
+------+--------------+-------------+------------+----------+------+-------+----------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+-----+
|     1|         29.85|            0|       29.85|7590-VHVEG|Female|    Yes|        No|          No|No phone service|            DSL|            No|         Yes|     

# 6. Feature-инжиниринг
Добавьте в датафрейм одну или несколько новых фичей. Удалите колонки, которые, как вам кажется, нужно убрать из фичей. Обоснуйте свои решения.

In [119]:
from pyspark.sql.functions import col, when

# Добавление новых фичей
df_feat = df_cleaned.withColumn("tenure_months", col("tenure") * 12) \
       .withColumn("TotChrg_pmnth", col("TotalCharges") / col("tenure")) \

# Удаление ненужных колонок
df_feat = df_feat.drop("tenure", "TotalCharges", "customerID")
df_feat.show(5)


+--------------+-------------+------+-------+----------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+-----+-------------+-----------------+
|MonthlyCharges|SeniorCitizen|gender|Partner|Dependents|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|Churn|tenure_months|    TotChrg_pmnth|
+--------------+-------------+------+-------+----------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+-----+-------------+-----------------+
|         29.85|            0|Female|    Yes|        No|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

#7. Векторизация фичей
Подготовьте данные к обучению:





1. Преобразуйте текстовые колонки в числа, используя StringIndexer.
Удалите столбцы со старыми (непреобразованными) признаками. Выведите на экран структуру получившегося датафрейма. Не забывайте о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [120]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#список колонок с текстовым типом
text_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "Churn"]


indexers = [StringIndexer(inputCol=column, outputCol=column+"_xd").fit(df_feat) for column in text_cols]
pipeline = Pipeline(stages=indexers)
df_numerated = pipeline.fit(df_feat).transform(df_feat).drop(*text_cols)
df_numerated.show(5)

+--------------+-------------+-------------+-----------------+---------+----------+-------------+---------------+----------------+------------------+-----------------+---------------+-------------------+--------------+--------------+------------------+-----------+-------------------+----------------+--------+
|MonthlyCharges|SeniorCitizen|tenure_months|    TotChrg_pmnth|gender_xd|Partner_xd|Dependents_xd|PhoneService_xd|MultipleLines_xd|InternetService_xd|OnlineSecurity_xd|OnlineBackup_xd|DeviceProtection_xd|TechSupport_xd|StreamingTV_xd|StreamingMovies_xd|Contract_xd|PaperlessBilling_xd|PaymentMethod_xd|Churn_xd|
+--------------+-------------+-------------+-----------------+---------+----------+-------------+---------------+----------------+------------------+-----------------+---------------+-------------------+--------------+--------------+------------------+-----------+-------------------+----------------+--------+
|         29.85|            0|           12|            29.85|     

2. Векторизуйте категориальные признаки, используя OneHotEncoder.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран структуру получившегося после преобразований датафрейма.


In [121]:
from pyspark.ml.feature import OneHotEncoder

#список категориальных колонок
features_inp  = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]

input_feats = [x + "_xd" for x in features_inp]
output_feats = [x + "_vec" for x in input_feats]

encoder = OneHotEncoder(inputCols=input_feats, outputCols=output_feats)

fitted_encoder = encoder.fit(df_numerated)
df_vec = fitted_encoder.transform(df_numerated).drop(*input_feats)
df_vec.printSchema()

root
 |-- MonthlyCharges: double (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- tenure_months: integer (nullable = true)
 |-- TotChrg_pmnth: double (nullable = true)
 |-- Churn_xd: double (nullable = false)
 |-- gender_xd_vec: vector (nullable = true)
 |-- Partner_xd_vec: vector (nullable = true)
 |-- Dependents_xd_vec: vector (nullable = true)
 |-- PhoneService_xd_vec: vector (nullable = true)
 |-- MultipleLines_xd_vec: vector (nullable = true)
 |-- InternetService_xd_vec: vector (nullable = true)
 |-- OnlineSecurity_xd_vec: vector (nullable = true)
 |-- OnlineBackup_xd_vec: vector (nullable = true)
 |-- DeviceProtection_xd_vec: vector (nullable = true)
 |-- TechSupport_xd_vec: vector (nullable = true)
 |-- StreamingTV_xd_vec: vector (nullable = true)
 |-- StreamingMovies_xd_vec: vector (nullable = true)
 |-- Contract_xd_vec: vector (nullable = true)
 |-- PaperlessBilling_xd_vec: vector (nullable = true)
 |-- PaymentMethod_xd_vec: vector (nullable = true)



3. Объедините колонки фичей в один вектор, используя VectorAssembler.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран первые несколько строк и структуру получившегося датафрейма.

In [122]:
from pyspark.ml.feature import VectorAssembler

X_list = df_vec.drop("Churn_xd").columns
target = "Churn_xd"

vectorizer = VectorAssembler(inputCols=X_list, outputCol="features_vec")
df_vectorized = vectorizer.transform(df_vec).drop(*X_list)
df_vectorized.show(5,200)

+--------+------------------------------------------------------------------------------------------------------------------------------------+
|Churn_xd|                                                                                                                        features_vec|
+--------+------------------------------------------------------------------------------------------------------------------------------------+
|     0.0|                         (30,[0,2,3,6,11,12,15,16,18,20,22,24,26,27],[29.85,12.0,29.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|     0.0|   (30,[0,2,3,4,5,6,7,8,11,13,14,17,18,20,22,28],[56.95,408.0,55.5735294117647,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|     1.0|(30,[0,2,3,4,5,6,7,8,11,13,15,16,18,20,22,24,26,28],[53.85,24.0,54.075,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|     0.0|               (30,[0,2,3,4,5,6,11,13,14,17,19,20,22,29],[42.3,540.0,40.90555555555556,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

#8. Создание и обучение модели

1. Создайте модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [123]:
from pyspark.ml.classification import LogisticRegression

logreg = LogisticRegression(featuresCol="features_vec", labelCol=target, predictionCol="y_pred")

2. Разделите датафрейм на обучающую и тестовую выборку.

In [124]:
X_train, X_test = df_vectorized.randomSplit([.8, .2])
X_train.show(5)

+--------+--------------------+
|Churn_xd|        features_vec|
+--------+--------------------+
|     0.0|(30,[0,1,2,3,4,5,...|
|     0.0|(30,[0,1,2,3,4,5,...|
|     0.0|(30,[0,1,2,3,4,5,...|
|     0.0|(30,[0,1,2,3,4,5,...|
|     0.0|(30,[0,1,2,3,4,5,...|
+--------+--------------------+
only showing top 5 rows



3. Создайте объекты — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. Так же, как и в ноутбуке из последнего видео, в сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

Вы можете ознакомиться [с документацией объекта LogisticRegression в PySpark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html) и добавить в сетку больше параметров.


In [125]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(logreg.maxIter, [10, 50, 100]) \
    .addGrid(logreg.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(logreg.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(logreg.family, ["auto", "binomial", "multinomial"]) \
    .build()



4. Создайте объект evaluator, который будет отвечать за метрику качества при обучении. Для этого используйте класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

У вас, возможно, возник вопрос, какую метрику качества берёт по умолчанию BinaryClassificationEvaluator. По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [126]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

bce = BinaryClassificationEvaluator(rawPredictionCol="y_pred", labelCol="Churn_xd")

5. Создайте объект CrossValidator, в качестве параметров укажите уже созданные вами модель, сетку гиперпараметров и evaluator.

In [127]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=logreg, estimatorParamMaps=paramGrid, evaluator=bce)

6. Запустите обучение модели на тренировочной выборке. Сохраните обученную модель в новую переменную.

In [128]:
cv_model = cv.fit(X_train)

#9. Выбор лучшей модели

1. Выберите лучшую модель, сохраните её в отдельную переменную, отобразите её параметры.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [130]:
best_model = cv_model.bestModel
best_model.extractParamMap()

{Param(parent='LogisticRegression_f57e280eea20', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_f57e280eea20', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_f57e280eea20', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'multinomial',
 Param(parent='LogisticRegression_f57e280eea20', name='featuresCol', doc='features column name.'): 'features_vec',
 Param(parent='LogisticRegression_f57e280eea20', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_f57e280eea20', name='labelCol', doc='label column name.'): 'Churn_xd',
 Param(parent='LogisticRegression_f57e280eea20', name='maxBlockSizeInMB', doc='maximum memo

2. Запустите лучшую модель в режиме предсказания на тестовой выборке. Сохраните предсказания в отдельную переменную. Выведите первые несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [131]:
y_test = best_model.transform(X_test)
y_test.show(5)

+--------+--------------------+--------------------+--------------------+------+
|Churn_xd|        features_vec|       rawPrediction|         probability|y_pred|
+--------+--------------------+--------------------+--------------------+------+
|     0.0|(30,[0,1,2,3,4,5,...|[0.08158622991016...|[0.54070284471500...|   0.0|
|     0.0|(30,[0,1,2,3,4,5,...|[0.26813208748351...|[0.63094294247736...|   0.0|
|     0.0|(30,[0,1,2,3,4,5,...|[2.45356344809640...|[0.99266056487297...|   0.0|
|     0.0|(30,[0,1,2,3,4,5,...|[1.35259927578399...|[0.93733270255373...|   0.0|
|     0.0|(30,[0,1,2,3,4,5,...|[1.16977315535992...|[0.91209971796692...|   0.0|
+--------+--------------------+--------------------+--------------------+------+
only showing top 5 rows



3. Получите метрику качества модели. Для этого примените к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).



In [132]:
bce.evaluate(y_test)

0.7044937672981613

#10. Обратная связь
Вы ознакомились с возможностями двух мощных библиотек: PySpark SQL для предобработки данных и PySpark ML для машинного обучения.

Поделитесь впечатлениями от работы с новыми библиотеками. В чём они более удобны, чем уже знакомые вам Pandas и Sklearn, а в чём нет.

# Как отправить работу на проверку

Загрузите файл с заданиями, откройте его через Jupyter Notebook в Google Colab. Скачайте файл с датасетом и загрузите его в Colab. Выполните задачи, сохраните изменения: воспользуйтесь опцией Save and Checkpoint из вкладки меню File или кнопкой Save and Checkpoint на панели инструментов. Отправьте через форму ниже итоговый файл Jupyter Notebook (.ipynb) или ссылку на него.